In [1]:
import sqlalchemy
sqlalchemy.__version__

'1.3.19'

# Define and Create Tables¶

In [2]:
from sqlalchemy import create_engine

'''
[The object created below]:
- represents the core interface to the database
- has not actually tried to connect to the database yet;
  that happens only the first time
  it is asked to perform a task against the database.

- is a repository for database connections
  capable of issuing SQL to the database.
'''
engine = create_engine('sqlite:///:memory:', echo=True)

In [3]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

metadata = MetaData()

users = Table(
    'users', metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String),
    Column('fullname', String),
)
addresses = Table(
    'addresses', metadata,
    Column('id', Integer, primary_key=True),
    Column('user_id', None, ForeignKey('users.id')),
    Column('email_address', String, nullable=False)
)

# actually ... create our ... tables ... inside the ... database
metadata.create_all(engine)

2020-09-30 17:37:43,769 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-09-30 17:37:43,771 INFO sqlalchemy.engine.base.Engine ()
2020-09-30 17:37:43,773 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-09-30 17:37:43,776 INFO sqlalchemy.engine.base.Engine ()
2020-09-30 17:37:43,783 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2020-09-30 17:37:43,788 INFO sqlalchemy.engine.base.Engine ()
2020-09-30 17:37:43,791 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2020-09-30 17:37:43,806 INFO sqlalchemy.engine.base.Engine ()
2020-09-30 17:37:43,817 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("addresses")
2020-09-30 17:37:43,818 INFO sqlalchemy.engine.base.Engine ()
2020-09-30 17:37:43,857 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("addresses")
2020-09-30 17:37:43,859 INFO sqlalchemy.engine.base.Engine ()
2020-09-30 17:37:4

# [The first SQL expression we’ll create is] the `Insert` construct

In [4]:
# the `Insert` construct ... represents an INSERT statement.
ins = users.insert()
str(ins)

'INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)'

In [5]:
ins = users.insert().values(name='jack', fullname='Jack Jones')
str(ins)

'INSERT INTO users (name, fullname) VALUES (:name, :fullname)'

In [6]:
ins.compile().params

{'name': 'jack', 'fullname': 'Jack Jones'}

# Executing [an `Insert` construct]

In [7]:
# The interesting part of an `Insert` construct is executing it.

# acquire a [database] connection
conn = engine.connect()
conn

In [8]:
result = conn.execute(ins)

print()
print(type(result), '...')

2020-09-30 17:37:44,112 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2020-09-30 17:37:44,120 INFO sqlalchemy.engine.base.Engine ('jack', 'Jack Jones')
2020-09-30 17:37:44,125 INFO sqlalchemy.engine.base.Engine COMMIT

<class 'sqlalchemy.engine.result.ResultProxy'> ...


... is analogous to "the DBAPI cursor object"

In [9]:
ins.bind = engine
str(ins)

'INSERT INTO users (name, fullname) VALUES (?, ?)'

In [10]:
result.inserted_primary_key

[1]

**NB**:
- The value of 1 was automatically generated by SQLite,
but only because we did not specify the `id` column in our `Insert`;
- otherwise, our explicit value would have been used.

# Executing Multiple Statements

In [11]:
# Our insert example above was intentionally a little drawn out
# to show some various behaviors of expression language constructs...
# Let's create a generic `Insert` ... again
# and use it in the “normal” way:
ins = users.insert()
conn.execute(
    ins,
    id=2, name='wendy', fullname='Wendy Williams'
)

2020-09-30 17:37:44,217 INFO sqlalchemy.engine.base.Engine INSERT INTO users (id, name, fullname) VALUES (?, ?, ?)
2020-09-30 17:37:44,263 INFO sqlalchemy.engine.base.Engine (2, 'wendy', 'Wendy Williams')
2020-09-30 17:37:44,273 INFO sqlalchemy.engine.base.Engine COMMIT


In [12]:
# To issue many inserts, ...
# - we can send in a list of dictionaries;
# - each dictionary *must* have the same set of keys.
conn.execute(
    addresses.insert(),
    [
        {'user_id': 1, 'email_address' : 'jack@yahoo.com'},
        {'user_id': 1, 'email_address' : 'jack@msn.com'},
        {'user_id': 2, 'email_address' : 'www@www.org'},
        {'user_id': 2, 'email_address' : 'wendy@aol.com'},
    ]
)

2020-09-30 17:37:44,335 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (user_id, email_address) VALUES (?, ?)
2020-09-30 17:37:44,341 INFO sqlalchemy.engine.base.Engine ((1, 'jack@yahoo.com'), (1, 'jack@msn.com'), (2, 'www@www.org'), (2, 'wendy@aol.com'))
2020-09-30 17:37:44,352 INFO sqlalchemy.engine.base.Engine COMMIT


# Selecting

In [13]:
# The primary construct used to generate SELECT statements
# is the `select()` function:
from sqlalchemy.sql import select

s = select([users])

In [14]:
result = conn.execute(s)

print()
print(type(result))

print()
for row in result:
    print(f'{type(row)}: {row}')

2020-09-30 17:37:44,399 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2020-09-30 17:37:44,407 INFO sqlalchemy.engine.base.Engine ()

<class 'sqlalchemy.engine.result.ResultProxy'>

<class 'sqlalchemy.engine.result.RowProxy'>: (1, 'jack', 'Jack Jones')
<class 'sqlalchemy.engine.result.RowProxy'>: (2, 'wendy', 'Wendy Williams')


- The `ResultProxy` acts much like a "DBAPI cursor", including methods such as `fetchone()` and `fetchall()`
- The `RowProxy` behaves like a hybrid between a mapping and tuple, with several methods for retrieving data from each column.

In [15]:
result = conn.execute(s)

row = result.fetchone()
print()
print(f'name: {row["name"]}; fullname: {row["fullname"]}')

row = result.fetchone()
print()
print(f'name: {row[1]}; fullname: {row[2]}')

2020-09-30 17:37:44,434 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2020-09-30 17:37:44,449 INFO sqlalchemy.engine.base.Engine ()

name: jack; fullname: Jack Jones

name: wendy; fullname: Wendy Williams


In [16]:
result = conn.execute(s)

for row in result:
    print()
    print(f'name: {row[users.c.name]}; fullname: {row[users.c.fullname]}')

2020-09-30 17:37:44,473 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2020-09-30 17:37:44,477 INFO sqlalchemy.engine.base.Engine ()

name: jack; fullname: Jack Jones

name: wendy; fullname: Wendy Williams


The `ResultProxy` object features “auto-close” behavior that closes the underlying "DBAPI cursor" object when all pending result rows have been fetched.

If a `ResultProxy` is to be discarded before such an autoclose has occurred, it can be explicitly closed...:
```
>>> result.close()
```

# Selecting Specific Columns

In [17]:
s = select([users.c.name, users.c.fullname])

result = conn.execute(s)

for row in result:
    print()
    print(row)

2020-09-30 17:37:44,494 INFO sqlalchemy.engine.base.Engine SELECT users.name, users.fullname 
FROM users
2020-09-30 17:37:44,500 INFO sqlalchemy.engine.base.Engine ()

('jack', 'Jack Jones')

('wendy', 'Wendy Williams')


In [18]:
# putting two tables into our `select()` statement ...
# is a Cartesian product;
# each row from the `users` table is produced
# against each row from the `addresses` table.

for row in conn.execute(
    select([users, addresses])
):
    print()
    print(row)

2020-09-30 17:37:44,523 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses
2020-09-30 17:37:44,526 INFO sqlalchemy.engine.base.Engine ()

(1, 'jack', 'Jack Jones', 1, 1, 'jack@yahoo.com')

(1, 'jack', 'Jack Jones', 2, 1, 'jack@msn.com')

(1, 'jack', 'Jack Jones', 3, 2, 'www@www.org')

(1, 'jack', 'Jack Jones', 4, 2, 'wendy@aol.com')

(2, 'wendy', 'Wendy Williams', 1, 1, 'jack@yahoo.com')

(2, 'wendy', 'Wendy Williams', 2, 1, 'jack@msn.com')

(2, 'wendy', 'Wendy Williams', 3, 2, 'www@www.org')

(2, 'wendy', 'Wendy Williams', 4, 2, 'wendy@aol.com')


In [19]:
# to put some sanity into this statement, we need a WHERE clause

s = select(
    [users, addresses]
).where(
    users.c.id == addresses.c.user_id
)

for row in conn.execute(s):
    print()
    print(row)

2020-09-30 17:37:44,550 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id
2020-09-30 17:37:44,557 INFO sqlalchemy.engine.base.Engine ()

(1, 'jack', 'Jack Jones', 1, 1, 'jack@yahoo.com')

(1, 'jack', 'Jack Jones', 2, 1, 'jack@msn.com')

(2, 'wendy', 'Wendy Williams', 3, 2, 'www@www.org')

(2, 'wendy', 'Wendy Williams', 4, 2, 'wendy@aol.com')


But let’s look at `users.c.id == addresses.c.id`? It’s using just a Python equality operator between two different `Column` objects.

In [20]:
users.c.id == addresses.c.user_id

In [21]:
str(users.c.id == addresses.c.user_id)

'users.id = addresses.user_id'

By now, one can see that everything we are working with is ultimately the same type of object. SQLAlchemy terms the base class of all of these expressions as `ColumnElement`.